In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv("Bank Customer Churn Prediction.csv")
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 12)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [5]:
df.isnull().sum()

,0
customer_id,0
credit_score,0
country,0
gender,0
age,0
tenure,0
balance,0
products_number,0
credit_card,0
active_member,0


In [6]:
df.duplicated().sum()

0

In [7]:
df.describe()

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])
df['gender'].unique()

array([0, 1])

In [9]:
df= pd.get_dummies(df, columns=['country'])


In [10]:
df.columns[df.columns.str.startswith('country_')].tolist()

['country_France', 'country_Germany', 'country_Spain']

In [11]:
scaler = StandardScaler()
df[['credit_score', 'age', 'balance', 'estimated_salary']] = scaler.fit_transform(
    df[['credit_score', 'age', 'balance', 'estimated_salary']])


In [12]:
churn_counts = df['churn'].value_counts()
churn_counts

,count
churn,
0,7963
1,2037


In [13]:
df

,customer_id,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,15634602,-0.326221,0,0.293517,2,-1.225848,1,1,1,0.021886,1,True,False,False
1,15647311,-0.440036,0,0.198164,1,0.117350,1,0,1,0.216534,0,False,False,True
2,15619304,-1.536794,0,0.293517,8,1.333053,3,1,0,0.240687,1,True,False,False
3,15701354,0.501521,0,0.007457,1,-1.225848,2,0,0,-0.108918,0,True,False,False
4,15737888,2.063884,0,0.388871,2,0.785728,1,1,1,-0.365276,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,1.246488,1,0.007457,5,-1.225848,2,1,0,-0.066419,0,True,False,False
9996,15569892,-1.391939,1,-0.373958,10,-0.306379,1,1,1,0.027988,0,True,False,False
9997,15584532,0.604988,0,-0.278604,7,-1.225848,1,0,1,-1.008643,1,True,False,False
9998,15682355,1.256835,1,0.293517,3,-0.022608,2,1,0,-0.125231,1,False,True,False


In [14]:
X=df.drop(['churn','customer_id'],axis=1)
y=df['churn']

In [15]:
# Convert all columns to float32
X = X.astype(np.float32)

In [16]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X_train

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain
9254,0.367013,1.0,-0.660019,6.0,-1.225848,2.0,1.0,1.0,1.373784,1.0,0.0,0.0
1561,-0.191713,1.0,0.293517,4.0,0.691389,2.0,1.0,1.0,1.667407,0.0,1.0,0.0
1670,-0.947028,1.0,-1.422847,3.0,0.613102,1.0,1.0,0.0,-0.246910,0.0,0.0,1.0
6087,-0.926334,0.0,-1.136786,9.0,0.948021,1.0,1.0,0.0,0.921446,1.0,0.0,0.0
6669,-1.381593,1.0,1.628468,9.0,1.052363,1.0,0.0,0.0,-1.053812,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,1.215448,1.0,1.437760,8.0,-0.108554,1.0,1.0,1.0,-0.534002,1.0,0.0,0.0
5191,0.325625,0.0,1.819175,1.0,-1.225848,1.0,1.0,1.0,-1.728184,1.0,0.0,0.0
5390,0.874005,0.0,-0.087897,1.0,-1.225848,3.0,0.0,0.0,-0.136854,1.0,0.0,0.0
860,0.170424,1.0,0.388871,8.0,1.822949,1.0,1.0,0.0,-0.044902,1.0,0.0,0.0


In [18]:
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from keras.callbacks import EarlyStopping


In [19]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.6/565.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00


In [20]:
# Install Git if needed (it should already be installed in Colab)
!apt-get install git

# Set up GitHub configuration
!git config --global user.name "user anme"
!git config --global user.email "email"
!git clone https://<tokan>@github.com/Talal16/DEPI--Project-01.git

%cd /content/DEPI--Project-01

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'DEPI--Project-01'...
remote: Enumerating objects: 496, done.
remote: Counting objects: 100% (496/496), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 496 (delta 70), reused 485 (delta 65), pack-reused 0 (from 0)
Receiving objects: 100% (496/496), 2.89 MiB | 16.16 MiB/s, done.
Resolving deltas: 100% (70/70), done.
/content/DEPI--Project-01


In [21]:
import mlflow
import os

# Path to the cloned GitHub repository
github_repo_dir = "/content/DEPI--Project-01/mlflow_results/"

# Create a directory for the MLflow artifacts inside your cloned repo
os.makedirs(github_repo_dir, exist_ok=True)


In [22]:
%cd /content/DEPI--Project-01/mlflow_results/

/content/DEPI--Project-01/mlflow_results


In [23]:
import mlflow
import mlflow.keras
import mlflow.sklearn
from mlflow.models import infer_signature
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [24]:
!pip install pyngrok --quiet

In [25]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5002 &") # run tracking UI in the background
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2nAIUiiMheuxBUULkyXA4MXrw3o_6Fxn8pQus3pttmsEm4MYn"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5002 for http://localhost:50002
ngrok_tunnel = ngrok.connect(addr="5002", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://053e-34-82-34-113.ngrok-free.app


In [26]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5002")

In [27]:
mlflow.set_experiment("Bank Customer Churn Prediction")

<Experiment: artifact_location='mlflow-artifacts:/155538848640535983', creation_time=1728658329223, experiment_id='155538848640535983', last_update_time=1728658329223, lifecycle_stage='active', name='Bank Customer Churn Prediction', tags={'mlflow.sharedViewState.4bc4f076692a45e46fb3e5fc464031e1108ac62b3457c0b3b576687717e6e5b0': 'deflate;eJxlUsFuGkEM/ZVozqgiV24bRFoUqKKFRJVQBc6Md9eSd2Y1nqHQiH+vF0gJ4ej3/Gw/2+9GEKJtHokTRjMyZmBCdBgf9k+417jFFMnKt80WeA3Zbi4JhVgzqoAFB0YSxLSkFlVSzGaaxFSh3VvG/6ULm2iLSjlIIJjkg1n9Hpg2OORXjELBXxTMd2X2ohpBRpvQjQPnVpHRyiSoda5lBPLk67upr0I/3fXE1uYIdn9LnKxcYRxEbsAuoqV+qhtGCWDu4Q4itIpaBpH18Ba63xj1GNXKZNeBd+jM6P1wQp7J+z5enTN+kHPov8Zz3Y9u5HFaLpbr++G6fPm50DZbwj9z2FFLf/sa52uobEaSPiqdUZLCWr2deinxeMNPkiz4PYbcoXsFzihTP270pLroFLPyDTmctF3aX8N1r3nQR/GZ+Rx+8Qg5hRKriNJMPLzxp6Y1hzfgGXk8Vh0HX1GtKrMrdiSnB5SE3fGdPC7aEFLjUbT98PISv/rk+fEyJ4k5HP4BDmQC4Q=='}>

In [28]:
def train_model( X_train, y_train, X_test, y_test,massage,epochs=100,class_weight={0:1, 1:1}):

    # Before calling model.fit, reset the index of X_train and y_train:
    X_train = X_train.reset_index(drop=True)  # Resets index to 0, 1, 2, ...
    y_train = y_train.reset_index(drop=True)  # Resets index to 0, 1, 2, ...
    X_test = X_test.reset_index(drop=True)  # Resets index to 0, 1, 2, ...
    y_test = y_test.reset_index(drop=True)  # Resets index to 0, 1, 2, ...

    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.36))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=0.00294),
        loss='binary_crossentropy',
        metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')]
    )
    early_stopping = EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)

    result=model.fit(X_train, y_train, epochs=epochs, batch_size=64,
                     validation_data=(X_test, y_test),
                     callbacks=[early_stopping],
                     class_weight=class_weight
                    )
     # Build the Evaluation Dataset from the test set
    eval_data = pd.DataFrame(X_test.copy())
    eval_data["label"] = y_test

    # Create a model signature
    X_test_np = X_test.to_numpy()
    signature = infer_signature(X_test_np, model.predict(X_test_np))


    with mlflow.start_run() as run:
        # Log the baseline model to MLflow
          # Set a tag that we can use to remind ourselves what this run was for
        mlflow.set_tag("Training Info",massage)
        mlflow.keras.log_model(model, "model", signature=signature ,registered_model_name=massage)
        mlflow.log_params({"class_0":class_weight[0],"class_1":class_weight[1]})
        mlflow.log_metric("val_loss", result.history['val_loss'][-1])
        mlflow.log_metric("val_accuracy", result.history['val_accuracy'][-1])
        mlflow.log_metric("val_precision", result.history['val_precision'][-1])
        mlflow.log_metric("val_recall", result.history['val_recall'][-1])
        mlflow.log_metric("val_auc", result.history['val_auc'][-1])






In [29]:
train_model(
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        massage="baseline",
        epochs=100,
    )

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.7823 - auc: 0.6425 - loss: 0.5053 - precision: 0.3962 - recall: 0.0866 - val_accuracy: 0.8180 - val_auc: 0.7913 - val_loss: 0.4075 - val_precision: 0.6747 - val_recall: 0.1425
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8113 - auc: 0.7579 - loss: 0.4413 - precision: 0.6702 - recall: 0.1976 - val_accuracy: 0.8260 - val_auc: 0.8014 - val_loss: 0.3993 - val_precision: 0.6243 - val_recall: 0.2875
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8283 - auc: 0.7868 - loss: 0.4180 - precision: 0.7002 - recall: 0.3086 - val_accuracy: 0.8370 - val_auc: 0.8087 - val_loss: 0.3889 - val_precision: 0.7965 - val_recall: 0.2290
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8389 - auc: 0.7991 - loss: 0.4008 - precision: 0.7282 - recall: 0.3491 - val_accuracy: 0.8420 - val_auc: 0.8198 - val_loss: 0.3759 - val_precision: 0.7484 - val_recall: 0.2952
Epoch 5/100
125/125 ━━━━

Registered model 'baseline' already exists. Creating a new version of this model...
2024/10/11 19:19:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: baseline, version 2
Created version '2' of model 'baseline'.
2024/10/11 19:19:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run shivering-snail-613 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/d25264258c024683a671269bd5585110.
2024/10/11 19:19:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


In [30]:
from sklearn.utils.class_weight import compute_class_weight
# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Print class weight for debugging
print("Class weights:", class_weight_dict)

Class weights: {0: 0.6293266205160478, 1: 2.4330900243309004}


In [31]:
train_model(X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            massage="class_weight using compute_class_weight as balanced",
            epochs=100,
            class_weight=class_weight_dict
            )

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6275 - auc: 0.6605 - loss: 0.6552 - precision: 0.2993 - recall: 0.5974 - val_accuracy: 0.7210 - val_auc: 0.7867 - val_loss: 0.5683 - val_precision: 0.3856 - val_recall: 0.7074
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7001 - auc: 0.7514 - loss: 0.6017 - precision: 0.3855 - recall: 0.6816 - val_accuracy: 0.7315 - val_auc: 0.7991 - val_loss: 0.5395 - val_precision: 0.3977 - val_recall: 0.7125
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7033 - auc: 0.7671 - loss: 0.5807 - precision: 0.3804 - recall: 0.6933 - val_accuracy: 0.7455 - val_auc: 0.8122 - val_loss: 0.5369 - val_precision: 0.4150 - val_recall: 0.7201
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7358 - auc: 0.8026 - loss: 0.5508 - precision: 0.4297 - recall: 0.7114 - val_accuracy: 0.7780 - val_auc: 0.8192 - val_loss: 0.5039 - val_precision: 0.4570 - val_recall: 0.6896
Epoch 5/100
125/125 ━━━━━━━━

Registered model 'class_weight using compute_class_weight as balanced' already exists. Creating a new version of this model...
2024/10/11 19:20:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: class_weight using compute_class_weight as balanced, version 2
Created version '2' of model 'class_weight using compute_class_weight as balanced'.
2024/10/11 19:20:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run righteous-mink-222 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/a402d97144bb4c539f1bdd769e0c7e27.
2024/10/11 19:20:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


In [32]:
for i in range(3, 9):
    class_weight_dict = {0: 1.0, 1: i}

    train_model(X_train=X_train,
                X_test=X_test,
                y_train=y_train,
                y_test=y_test,
                massage=f"change class_weight for class_1 {i}",
                epochs=100,
                class_weight=class_weight_dict
                )

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6773 - auc: 0.6590 - loss: 0.9303 - precision: 0.3262 - recall: 0.5032 - val_accuracy: 0.7715 - val_auc: 0.7901 - val_loss: 0.5194 - val_precision: 0.4422 - val_recall: 0.6234
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7419 - auc: 0.7589 - loss: 0.8190 - precision: 0.4162 - recall: 0.5980 - val_accuracy: 0.7860 - val_auc: 0.7984 - val_loss: 0.4834 - val_precision: 0.4678 - val_recall: 0.6463
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7631 - auc: 0.7844 - loss: 0.7806 - precision: 0.4421 - recall: 0.6165 - val_accuracy: 0.8065 - val_auc: 0.8162 - val_loss: 0.4923 - val_precision: 0.5061 - val_recall: 0.6361
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7833 - auc: 0.8015 - loss: 0.7487 - precision: 0.4736 - recall: 0.6423 - val_accuracy: 0.8270 - val_auc: 0.8215 - val_loss: 0.4200 - val_precision: 0.5655 - val_recall: 0.5165
Epoch 5/100
125/125 ━━━━━━━━

Registered model 'change class_weight for class_1 3' already exists. Creating a new version of this model...
2024/10/11 19:20:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: change class_weight for class_1 3, version 2
Created version '2' of model 'change class_weight for class_1 3'.
2024/10/11 19:20:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run whimsical-bee-460 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/8e5b16ace2384404a053b0ae5911c3fd.
2024/10/11 19:20:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6130 - auc: 0.6449 - loss: 1.0993 - precision: 0.2924 - recall: 0.6114 - val_accuracy: 0.7350 - val_auc: 0.7848 - val_loss: 0.5619 - val_precision: 0.4017 - val_recall: 0.7125
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6803 - auc: 0.7537 - loss: 0.9602 - precision: 0.3638 - recall: 0.7143 - val_accuracy: 0.7095 - val_auc: 0.7952 - val_loss: 0.5787 - val_precision: 0.3792 - val_recall: 0.7506
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7075 - auc: 0.7767 - loss: 0.9309 - precision: 0.3929 - recall: 0.7189 - val_accuracy: 0.7495 - val_auc: 0.8090 - val_loss: 0.5193 - val_precision: 0.4192 - val_recall: 0.7125
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7260 - auc: 0.7933 - loss: 0.8964 - precision: 0.4075 - recall: 0.7270 - val_accuracy: 0.7785 - val_auc: 0.8248 - val_loss: 0.5019 - val_precision: 0.4579 - val_recall: 0.6921
Epoch 5/100
125/125 ━━━━━━━━

Registered model 'change class_weight for class_1 4' already exists. Creating a new version of this model...
2024/10/11 19:21:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: change class_weight for class_1 4, version 2
Created version '2' of model 'change class_weight for class_1 4'.
2024/10/11 19:21:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run worried-koi-772 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/d8b07ab8effd4118a2ba870859eaf3e5.
2024/10/11 19:21:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5726 - auc: 0.6730 - loss: 1.1846 - precision: 0.2851 - recall: 0.7007 - val_accuracy: 0.6745 - val_auc: 0.7848 - val_loss: 0.6268 - val_precision: 0.3510 - val_recall: 0.7735
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6615 - auc: 0.7588 - loss: 1.0477 - precision: 0.3439 - recall: 0.7726 - val_accuracy: 0.6765 - val_auc: 0.7894 - val_loss: 0.6457 - val_precision: 0.3527 - val_recall: 0.7735
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6723 - auc: 0.7794 - loss: 1.0340 - precision: 0.3679 - recall: 0.7524 - val_accuracy: 0.7460 - val_auc: 0.8109 - val_loss: 0.5253 - val_precision: 0.4168 - val_recall: 0.7328
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6859 - auc: 0.7876 - loss: 1.0074 - precision: 0.3736 - recall: 0.7785 - val_accuracy: 0.6665 - val_auc: 0.8243 - val_loss: 0.6404 - val_precision: 0.3511 - val_recall: 0.8219
Epoch 5/100
125/125 ━━━━━━━━

...

Registered model 'change class_weight for class_1 6' already exists. Creating a new version of this model...
2024/10/11 19:22:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: change class_weight for class_1 6, version 2
Created version '2' of model 'change class_weight for class_1 6'.
2024/10/11 19:22:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run abundant-wren-245 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/b1345dc2fa00484cad12a6a7f3e36594.
2024/10/11 19:22:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4785 - auc: 0.6654 - loss: 1.3888 - precision: 0.2627 - recall: 0.8341 - val_accuracy: 0.5850 - val_auc: 0.7875 - val_loss: 0.7106 - val_precision: 0.3044 - val_recall: 0.8651
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5778 - auc: 0.7580 - loss: 1.2374 - precision: 0.3041 - recall: 0.8485 - val_accuracy: 0.5655 - val_auc: 0.7957 - val_loss: 0.7564 - val_precision: 0.2945 - val_recall: 0.8677
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5777 - auc: 0.7761 - loss: 1.2140 - precision: 0.3174 - recall: 0.8447 - val_accuracy: 0.5545 - val_auc: 0.8096 - val_loss: 0.7331 - val_precision: 0.2908 - val_recall: 0.8804
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - auc: 0.7807 - loss: 1.1911 - precision: 0.3015 - recall: 0.8549 - val_accuracy: 0.5910 - val_auc: 0.8216 - val_loss: 0.7355 - val_precision: 0.3087 - val_recall: 0.8728
Epoch 5/100
125/125 ━━━━━━━━

Registered model 'change class_weight for class_1 7' already exists. Creating a new version of this model...
2024/10/11 19:22:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: change class_weight for class_1 7, version 2
Created version '2' of model 'change class_weight for class_1 7'.
2024/10/11 19:22:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run unleashed-zebra-181 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/66c29c09b3f94007b28c68d709163628.
2024/10/11 19:22:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4221 - auc: 0.6047 - loss: 1.6321 - precision: 0.2341 - recall: 0.8109 - val_accuracy: 0.4315 - val_auc: 0.7770 - val_loss: 0.9104 - val_precision: 0.2490 - val_recall: 0.9389
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5248 - auc: 0.7498 - loss: 1.3224 - precision: 0.2811 - recall: 0.8772 - val_accuracy: 0.5820 - val_auc: 0.7923 - val_loss: 0.7172 - val_precision: 0.3017 - val_recall: 0.8575
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5387 - auc: 0.7630 - loss: 1.2989 - precision: 0.2888 - recall: 0.8776 - val_accuracy: 0.5805 - val_auc: 0.8002 - val_loss: 0.7254 - val_precision: 0.3044 - val_recall: 0.8830
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5676 - auc: 0.7918 - loss: 1.2324 - precision: 0.3019 - recall: 0.8737 - val_accuracy: 0.5835 - val_auc: 0.8187 - val_loss: 0.7220 - val_precision: 0.3032 - val_recall: 0.8626
Epoch 5/100
125/125 ━━━━━━━━

Registered model 'change class_weight for class_1 8' already exists. Creating a new version of this model...
2024/10/11 19:23:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: change class_weight for class_1 8, version 2
Created version '2' of model 'change class_weight for class_1 8'.
2024/10/11 19:23:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run crawling-cod-87 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/f7366244c1f54d39abb397b61cdd5163.
2024/10/11 19:23:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


In [33]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
y_train_smote.value_counts()

,count
churn,
0,6356
1,6356


In [34]:
train_model(
        X_train=X_train_smote,
        y_train=y_train_smote,
        X_test=X_test,
        y_test=y_test,
        massage="smote",
        epochs=100,
    )

Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6473 - auc: 0.6951 - loss: 0.6390 - precision: 0.6461 - recall: 0.6356 - val_accuracy: 0.7305 - val_auc: 0.7927 - val_loss: 0.5459 - val_precision: 0.3978 - val_recall: 0.7226
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7250 - auc: 0.7974 - loss: 0.5490 - precision: 0.7259 - recall: 0.7142 - val_accuracy: 0.7435 - val_auc: 0.8194 - val_loss: 0.5325 - val_precision: 0.4155 - val_recall: 0.7506
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7476 - auc: 0.8243 - loss: 0.5150 - precision: 0.7547 - recall: 0.7404 - val_accuracy: 0.7915 - val_auc: 0.8345 - val_loss: 0.4667 - val_precision: 0.4790 - val_recall: 0.6972
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7599 - auc: 0.8423 - loss: 0.4929 - precision: 0.7604 - recall: 0.7536 - val_accuracy: 0.7375 - val_auc: 0.8469 - val_loss: 0.5393 - val_precision: 0.4136 - val_recall: 0.8041
Epoch 5/100
199/199 ━━━━━━━━

Registered model 'smote' already exists. Creating a new version of this model...
2024/10/11 19:23:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote, version 2
Created version '2' of model 'smote'.
2024/10/11 19:23:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run merciful-kit-129 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/ab58ab02bfcd42509a16437692256dbc.
2024/10/11 19:23:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


In [35]:
for i in range(2, 9):
    class_weight_dict = {0:1, 1: i}
    train_model(
        X_train=X_train_smote,
        y_train=y_train_smote,
        X_test=X_test,
        y_test=y_test,
        massage=f"smote with class_weight {i}",
        epochs=100,
        class_weight=class_weight_dict
    )

Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6026 - auc: 0.6775 - loss: 0.9090 - precision: 0.5665 - recall: 0.8482 - val_accuracy: 0.6415 - val_auc: 0.8012 - val_loss: 0.6876 - val_precision: 0.3350 - val_recall: 0.8372
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6985 - auc: 0.8008 - loss: 0.7583 - precision: 0.6481 - recall: 0.8667 - val_accuracy: 0.6550 - val_auc: 0.8285 - val_loss: 0.6790 - val_precision: 0.3464 - val_recall: 0.8524
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7285 - auc: 0.8355 - loss: 0.6960 - precision: 0.6778 - recall: 0.8722 - val_accuracy: 0.6655 - val_auc: 0.8447 - val_loss: 0.6558 - val_precision: 0.3559 - val_recall: 0.8677
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7505 - auc: 0.8559 - loss: 0.6581 - precision: 0.6964 - recall: 0.8867 - val_accuracy: 0.6890 - val_auc: 0.8542 - val_loss: 0.5999 - val_precision: 0.3735 - val_recall: 0.8601
Epoch 5/100
199/199 ━━━━━━━━

Registered model 'smote with class_weight 2' already exists. Creating a new version of this model...
2024/10/11 19:24:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote with class_weight 2, version 2
Created version '2' of model 'smote with class_weight 2'.
2024/10/11 19:24:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run spiffy-snipe-652 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/9ecac55ee87d4d8e9dc1d437c6af55b1.
2024/10/11 19:24:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5671 - auc: 0.6716 - loss: 1.1085 - precision: 0.5403 - recall: 0.9028 - val_accuracy: 0.3905 - val_auc: 0.7882 - val_loss: 0.9606 - val_precision: 0.2379 - val_recall: 0.9542
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6573 - auc: 0.8001 - loss: 0.8961 - precision: 0.6052 - recall: 0.9376 - val_accuracy: 0.5455 - val_auc: 0.8253 - val_loss: 0.8426 - val_precision: 0.2909 - val_recall: 0.9135
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6951 - auc: 0.8255 - loss: 0.8427 - precision: 0.6329 - recall: 0.9353 - val_accuracy: 0.5945 - val_auc: 0.8408 - val_loss: 0.7828 - val_precision: 0.3141 - val_recall: 0.8982
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7077 - auc: 0.8456 - loss: 0.8056 - precision: 0.6410 - recall: 0.9294 - val_accuracy: 0.5925 - val_auc: 0.8451 - val_loss: 0.7322 - val_precision: 0.3146 - val_recall: 0.9109
Epoch 5/100
199/199 ━━━━━━━

Registered model 'smote with class_weight 3' already exists. Creating a new version of this model...
2024/10/11 19:24:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote with class_weight 3, version 2
Created version '2' of model 'smote with class_weight 3'.
2024/10/11 19:24:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run wistful-finch-904 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/249f3319e48d414c8d2a301e2a70a848.
2024/10/11 19:24:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5459 - auc: 0.6609 - loss: 1.2462 - precision: 0.5268 - recall: 0.9391 - val_accuracy: 0.3570 - val_auc: 0.7919 - val_loss: 1.0620 - val_precision: 0.2302 - val_recall: 0.9695
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6225 - auc: 0.7933 - loss: 1.0156 - precision: 0.5748 - recall: 0.9594 - val_accuracy: 0.3770 - val_auc: 0.8218 - val_loss: 1.0198 - val_precision: 0.2366 - val_recall: 0.9746
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6495 - auc: 0.8267 - loss: 0.9437 - precision: 0.5974 - recall: 0.9617 - val_accuracy: 0.4335 - val_auc: 0.8312 - val_loss: 0.9604 - val_precision: 0.2540 - val_recall: 0.9720
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6663 - auc: 0.8422 - loss: 0.9149 - precision: 0.6062 - recall: 0.9620 - val_accuracy: 0.5405 - val_auc: 0.8384 - val_loss: 0.8695 - val_precision: 0.2896 - val_recall: 0.9211
Epoch 5/100
199/199 ━━━━━━━━

...

Registered model 'smote with class_weight 6' already exists. Creating a new version of this model...
2024/10/11 19:26:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote with class_weight 6, version 2
Created version '2' of model 'smote with class_weight 6'.
2024/10/11 19:26:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run orderly-hound-951 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/c9f3ff5fe64c4828a882233f4e44e78e.
2024/10/11 19:26:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5106 - auc: 0.6472 - loss: 1.5780 - precision: 0.5017 - recall: 0.9722 - val_accuracy: 0.3135 - val_auc: 0.7914 - val_loss: 1.2691 - val_precision: 0.2206 - val_recall: 0.9847
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5554 - auc: 0.7824 - loss: 1.2658 - precision: 0.5310 - recall: 0.9874 - val_accuracy: 0.3805 - val_auc: 0.8181 - val_loss: 1.1131 - val_precision: 0.2392 - val_recall: 0.9873
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5953 - auc: 0.8272 - loss: 1.1591 - precision: 0.5528 - recall: 0.9883 - val_accuracy: 0.4085 - val_auc: 0.8321 - val_loss: 1.1183 - val_precision: 0.2461 - val_recall: 0.9746
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6156 - auc: 0.8405 - loss: 1.1219 - precision: 0.5702 - recall: 0.9835 - val_accuracy: 0.3715 - val_auc: 0.8471 - val_loss: 1.2440 - val_precision: 0.2359 - val_recall: 0.9822
Epoch 5/100
199/199 ━━━━━━━━

Registered model 'smote with class_weight 7' already exists. Creating a new version of this model...
2024/10/11 19:26:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote with class_weight 7, version 2
Created version '2' of model 'smote with class_weight 7'.
2024/10/11 19:26:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run traveling-pug-510 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/a30c7685360243d08972b4ee30e45070.
2024/10/11 19:26:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5108 - auc: 0.6556 - loss: 1.5896 - precision: 0.5063 - recall: 0.9908 - val_accuracy: 0.2660 - val_auc: 0.7914 - val_loss: 1.4042 - val_precision: 0.2102 - val_recall: 0.9924
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5442 - auc: 0.7866 - loss: 1.3156 - precision: 0.5222 - recall: 0.9905 - val_accuracy: 0.2370 - val_auc: 0.8230 - val_loss: 1.2798 - val_precision: 0.2042 - val_recall: 0.9949
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5625 - auc: 0.8229 - loss: 1.2288 - precision: 0.5303 - recall: 0.9903 - val_accuracy: 0.2805 - val_auc: 0.8350 - val_loss: 1.4545 - val_precision: 0.2136 - val_recall: 0.9924
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5942 - auc: 0.8423 - loss: 1.1725 - precision: 0.5519 - recall: 0.9900 - val_accuracy: 0.3475 - val_auc: 0.8478 - val_loss: 1.2517 - val_precision: 0.2299 - val_recall: 0.9873
Epoch 5/100
199/199 ━━━━━━━━

Registered model 'smote with class_weight 8' already exists. Creating a new version of this model...
2024/10/11 19:27:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote with class_weight 8, version 2
Created version '2' of model 'smote with class_weight 8'.
2024/10/11 19:27:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run enthused-duck-437 at: http://127.0.0.1:5002/#/experiments/155538848640535983/runs/232d19c106f144fa9c462512e17702a2.
2024/10/11 19:27:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5002/#/experiments/155538848640535983.


In [38]:
import mlflow.keras
import tensorflow as tf

# Load the model as a PyFuncModel
loaded_model_class_1_6 = mlflow.pyfunc.load_model("mlflow-artifacts:/155538848640535983/a05734c6697f4759b12396b7b265bac4/artifacts/model")

# Access the underlying Keras model by calling `loaded_model_class_1_8.keras_model`
keras_model = loaded_model_class_1_6.get_raw_model()
print(keras_model)
# Save the Keras model in .h5 format
keras_model.save("/content/DEPI--Project-01/best_models/keras_model_class_1_6.keras")

<Sequential name=sequential_7, built=True>


In [37]:
# Load the model as a PyFuncModel
loaded_model_baseline= mlflow.pyfunc.load_model("mlflow-artifacts:/155538848640535983/d25264258c024683a671269bd5585110/artifacts/model")

# Access the underlying Keras model by calling `loaded_model_class_1_8.keras_model`
keras_model = loaded_model_baseline.get_raw_model()
print(keras_model)
# Save the Keras model in .h5 format
keras_model.save("/content/DEPI--Project-01/best_models/keras_model_baseline.keras")

<Sequential name=sequential, built=True>


In [40]:
# Load the model as a PyFuncModel
loaded_model_class_1_2= mlflow.pyfunc.load_model("mlflow-artifacts:/155538848640535983/256dea13183247fab42b62b6cba44ff8/artifacts/model")

# Access the underlying Keras model by calling `loaded_model_class_1_8.keras_model`
keras_model = loaded_model_class_1_2.get_raw_model()
print(keras_model)
# Save the Keras model in .h5 format
keras_model.save("/content/DEPI--Project-01/best_models/keras_model_class_1_2.keras")

<Sequential name=sequential_3, built=True>


In [41]:
%cd /content/DEPI--Project-01/

/content/DEPI--Project-01


In [42]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   mlflow_results/mlruns/models/baseline/meta.yaml
	modified:   mlflow_results/mlruns/models/change class_weight for class_1 3/meta.yaml
	modified:   mlflow_results/mlruns/models/change class_weight for class_1 4/meta.yaml
	modified:   mlflow_results/mlruns/models/change class_weight for class_1 5/meta.yaml
	modified:   mlflow_results/mlruns/models/change class_weight for class_1 6/meta.yaml
	modified:   mlflow_results/mlruns/models/change class_weight for class_1 7/meta.yaml
	modified:   mlflow_results/mlruns/models/change class_weight for class_1 8/meta.yaml
	modified:   mlflow_results/mlruns/models/class_weight using compute_class_weight as balanced/meta.yaml
	modified:   mlflow_results/mlruns/models/smote with class_weight 2/meta.yaml
	modified:   ml

In [43]:
!git add .

In [44]:
!git commit -m "add data set and MLflow model (NN)"

[main b27072e] add data set and MLflow model (NN)
 355 files changed, 1424 insertions(+), 16 deletions(-)
 create mode 100644 best_models/keras_model_baseline.keras
 create mode 100644 best_models/keras_model_class_1_2.keras
 create mode 100644 best_models/keras_model_class_1_6.keras
 create mode 100644 mlflow_results/mlartifacts/155538848640535983/20640b97dbf048849155c7b46748f59d/artifacts/model/MLmodel
 create mode 100644 mlflow_results/mlartifacts/155538848640535983/20640b97dbf048849155c7b46748f59d/artifacts/model/conda.yaml
 create mode 100644 mlflow_results/mlartifacts/155538848640535983/20640b97dbf048849155c7b46748f59d/artifacts/model/data/keras_module.txt
 create mode 100644 mlflow_results/mlartifacts/155538848640535983/20640b97dbf048849155c7b46748f59d/artifacts/model/data/model.keras
 create mode 100644 mlflow_results/mlartifacts/155538848640535983/20640b97dbf048849155c7b46748f59d/artifacts/model/python_env.yaml
 create mode 100644 mlflow_results/mlartifacts/155538848640535983/

In [45]:
!git push     # Push the new branch to main

Enumerating objects: 436, done.
Counting objects: 100% (436/436), done.
Delta compression using up to 2 threads
Compressing objects: 100% (223/223), done.
Writing objects: 100% (397/397), 2.03 MiB | 3.48 MiB/s, done.
Total 397 (delta 65), reused 42 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (65/65), completed with 18 local objects.
To https://github.com/Talal16/DEPI--Project-01.git
   79f8e01..b27072e  main -> main
